In [1]:
import numpy as np
import scipy.linalg as sla

In [2]:
dataset = {
    'A': np.array([
        [ 1., 2., 0., 1., 0.,  4., -1., -3.],
        [ 1., 3., 0., 0., 1., -1., -1.,  2.],
        [ 1., 4., 1., 0., 0.,  2., -2.,  0.],
    ]),
    'b': np.array([4., 5., 6.]),
    'B': np.array([
        [ 1., 1., -1., 0.,  3.,  4., -2.,  1.],
        [ 2., 6.,  0., 0.,  1., -5.,  0., -1.],
        [-1., 2.,  0., 0., -1.,  1.,  1.,  1.],
    ]),
    'd': np.array([7., 3., 3.]),
    'J': [2, 3, 4],
}
init_plan = np.array([0., 0., 6., 4., 5., 0., 0., 0.])
eps=1e-9

In [15]:
def solve(dataset, init_plan, verbose=True, verbose_iter=2, max_iter=1000):
    A = dataset['A'].copy()
    b = dataset['b'].copy()
    if 'D' not in dataset:
        B = dataset['B'].copy()
        D = B.T @ B
    else:
        D = dataset['D'].copy()
    if 'c' not in dataset:
        d = dataset['d'].copy()
        B = dataset['B'].copy()
        c = -d @ B
    else:
        c = dataset['c'].copy()
    J_base = [i for i in dataset['J']]
    J_ast = [i for i in dataset['J']]
    Jn = [i for i in range(A.shape[1]) if i not in J_base]
    x = init_plan.copy()
    
    if verbose:
        print('\n\n Initial data \n\n')
        print(f'Initial plan: {x}')
        print(f'A = {A}')
        print(f'b = {b}')
        print(f'D = {D}')
        print(f'c = {c}')
        print(f'J_base: {J_base}')
        print(f'J*: {J_ast}')
    
    skip_to_step3 = False
    
    for it in range(max_iter):
        
        if it >= verbose_iter:
            verbose = False
        
        if verbose:
            print('\n===========================')
            print(f'\nIteration {it + 1}\n')
        if not skip_to_step3:

            # Step 1
            c_ = D @ x + c
            A_base = A[:, J_base]
            c_base = c_[J_base]
            u = -c_base @ sla.inv(A_base)
            approx = u.T @ A + c_

            if verbose:
                print(f'c_: {c_}')
                print(f'A base: {A_base}')
                print(f'c base: {c_base}')
                print(f'u: {u}')
                print(f'Delta: {approx}')

            # Step 2
            if np.all(approx[Jn] > -eps):
                print(f'Optimal plan found: {x}')
                print(f'Iterations: {it}')
                print(f'cx = {c @ x + 0.5 * x @ D @ x}')
                return x

            j0 = np.argmin(approx)

            if verbose:
                print(f'j0: {j0}')
        
        skip_to_step3 = False
        
        # Step 3
        l = np.zeros(c.shape)
        l[j0] = 1.
        idxes = []
        D_ast = np.zeros((len(J_ast), len(J_ast)))
        A_ast = np.zeros((A.shape[0], len(J_ast)))
        for i in range(len(J_ast)):
            for j in range(len(J_ast)):
                D_ast[i, j] = D[J_ast[i], J_ast[j]]
        
        for j in range(len(J_ast)):
            A_ast[:, j] = A[:, J_ast[j]]
        
        H = np.concatenate((np.concatenate((D_ast, np.transpose(A_ast)), axis=1), 
                            np.concatenate((A_ast, np.zeros(A_base.shape)), axis=1)), axis=0)
        bb = np.concatenate((D[J_ast, j0], A[:, j0]), axis=0)
        print(bb)
        buffer = -sla.inv(H) @ bb
        l[J_ast] = buffer[:len(J_ast)]
        
        if verbose:
            print(f'l: {l}')
        
        # Step 4
        delta = l @ D @ l
        theta = []
        for i in J_ast:
            theta.append(np.inf if l[i] > -eps else -x[i] / l[i])
        
        theta.append(np.inf if delta < eps else np.abs(approx[j0]) / delta)
        theta = np.array(theta)
        j_ast = np.argmin(theta)
        if j_ast < theta.shape[0] - 1:
            j_ast = J_ast[j_ast]
        else:
            j_ast = j0
        theta0 = np.min(theta)
        
        if verbose:
            print(f'theta: {theta}')
            print(f'theta0: {theta0}')
            print(f'j*: {j_ast}')
            
        if theta0 == np.inf:
            print('\nNo solution\n')
            return
        
        # Step 5
        x_new = x + theta0 * l
        if verbose:
            print(f'x_new: {x_new}')
        
        # Step 6
        if j_ast == j0:
            J_ast.append(j0)
        elif j_ast in J_ast and j_ast not in J_base:
            approx[j0] += delta
            J_ast.remove(j_ast)
            skip_to_step3 = True
        else:
            J_ = [i for i in J_ast if i not in J_base]
            s = J_base.index(j_ast)
            flag = True
            e_s = np.zeros(len(J_base))
            e_s[s] = 1.
            for j_ in J_:
                if np.abs(e_s @ A_base @ A[:, j_]) > eps:
                    flag = False
                    break
            
            J_ast.remove(j_ast)
            J_base.remove(j_ast)
            if not flag:
                J_base.append(j_)
                approx[j0] += delta * theta0
                skip_to_step3 = True
            else:
                J_base.append(j0)
                J_ast.append(j0)
        
        x = x_new
        
        if verbose:
            print(f'J_base: {J_base}')
            print(f'J_ast: {J_ast}')
        


In [4]:
_ = solve(dataset, init_plan)



 Initial data 


Initial plan: [0. 0. 6. 4. 5. 0. 0. 0.]
A = [[ 1.  2.  0.  1.  0.  4. -1. -3.]
 [ 1.  3.  0.  0.  1. -1. -1.  2.]
 [ 1.  4.  1.  0.  0.  2. -2.  0.]]
b = [4. 5. 6.]
B = [[  6.  11.  -1.   0.   6.  -7.  -3.  -2.]
 [ 11.  41.  -1.   0.   7. -24.   0.  -3.]
 [ -1.  -1.   1.   0.  -3.  -4.   2.  -1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  6.   7.  -3.   0.  11.   6.  -7.   1.]
 [ -7. -24.  -4.   0.   6.  42.  -7.  10.]
 [ -3.   0.   2.   0.  -7.  -7.   5.  -1.]
 [ -2.  -3.  -1.   0.   1.  10.  -1.   3.]]
c = [-10. -31.   7.   0. -21. -16.  11.  -7.]
J_base: [2, 3, 4]
J*: [2, 3, 4]


Iteration 1

c_: [ 14.  -2.  -2.   0.  16. -10. -12.  -8.]
A base: [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
c base: [-2.  0. 16.]
u: [  0. -16.   2.]
Delta: [  0. -42.   0.   0.   0.  10.   0. -40.]
j0: 1
[-1.  0.  7.  2.  3.  4.]
l: [ 0.  1. -4. -2. -3.  0.  0.  0.]
theta: [1.5        2.         1.66666667 0.84      ]
theta0: 0.84
j*: 1
x_new: [0.   0.84 2.64 2.32 2.48 0.   0.   0.  ]
J_

In [36]:
# dataset2 = {
#     'A': np.array([[1, 2, 0, 1, 0, 4, -1, -3],
# [1, 3, 0, 0, 1, -1, -1, 2],
# [1, 4, 1, 0, 0, 2, -2, 0]]),
# 'b': np.array([4., 5, 6]),
# 'B': np.array([[1, 1, -1, 0, 3, 4, -2, 1],
# [2, 6, 0, 0, 1, -5, 0, -1],
# [-1, 2, 0, 0, -1, 1, 1, 1]]),
# 'd': np.array([7, 3, 3]),
# 'J': [3, 4, 5],
# }


# dataset2 = {
    
# 'A': np.array([[11,0,0,1,0,-4,-1,1],
# [1,1,0,0,1,-1,-1,1],
# [1,1,1,0,1,2,-2,1]]),
# 'b': np.array([8,2,5]),
# 'B': np.array([[1,-1,0,3,-1,5,-2,1],
# [2,5,0,0,-1,4,0,0],
# [-1,3,0,5,4,-1,-2,1]]),
# 'd': np.array([6,10,9]),
# 'J': [0, 1, 2]
# }
# init_plan2 = np.array([0.7921,1.2576,1.3811,1.1526,0.1258,0.5634,0.0713,0.4592])
# solve(dataset2, init_plan2, verbose=True)

# dataset2 = {
#     'A': np.array([[2,-3,1,1,3,0,1,2],
# [-1,3,1,0,1,4,5,-6],
# [1,1,-1,0,1,-2,4,8]]),
# 'b': np.array([8,4,14]),
# 'B': np.array([[1,0,0,3,-1,5,0,1],
# [2,5,0,0,0,4,0,0],
# [-1,9,0,5,2,-1,-1,5]]),
# 'c': np.array([-13,-217,0,-117,-27,-71,18,-99]),
# 'J': [1, 4, 7],
# }
# init_plan = np.array([0., 2, 0, 0, 4, 0, 0, 1])
# solve(dataset2, init_plan2, verbose=True)


# task 3
# dataset2 = {
#     'A': np.array([[0,2,1,4,3,0,-5,-10],
# [-1,3,1,0,1,3,-5,-6],
# [1,1,1,0,1,-2,-5,8]]),
# 'b': np.array([6,4,14]),
# 'D': np.array([[1,0,0,0,0,0,0,0],
# [0,1,0,0,0,0,0,0],
# [0,0,0,0,0,0,0,0],
# [0,0,0,1,0,0,0,0],
# [0,0,0,0,1,0,0,0],
# [0,0,0,0,0,1,0,0],
# [0,0,0,0,0,0,0,0],
# [0,0,0,0,0,0,0,1]]),
# 'c': np.array([1, 3, -1, 3, 5, 2, -2, 0]),
# 'J': [1, 4, 7]
# }

# init_plan2 = np.array([0,2,0,0,4,0,0,1])
# solve(dataset2, init_plan2)

# task 4
# dataset2 = {
#    'A': np.array([[0,2,1,4,3,0,-5,-10],
# [-1,1,1,0,1,1,-1,-1],
# [1,1,1,0,1,-2,-5,8]]),
# 'b': np.array([20,1,7]),
# 'D': np.array([[25,10,0,3,-1,13,0,1],
# [10,45,0,0,0,20,0,0],
# [0,0,20,0,0,0,0,0],
# [3,0,0,29,-3,15,0,3],
# [-1,0,0,-3,21,-5,0,-1],
# [13,20,0,15,-5,61,0,5],
# [0,0,0,0,0,0,20,0],
# [1,0,0,3,-1,5,0,21]]),
# 'c': np.array([1,-3,4,3,5,6,-2,0]),
# 'J': [0, 3, 4],
# }


# init_plan2 = np.array([3,0,0,2,4,0,0,0])
# solve(dataset2, init_plan2)

# task 5
dataset2 = {
    'A': np.array([[0,0,1,5,2,0,-5,-4],
[1,1,-1,0,1,-1,-1,-1],
[1,1,1,0,1,2,5,8]]),
'b': np.array([15,-1,9]),
'D': np.array([[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0]]),
'c': np.array([1,-3,4,3,5,6,-2,0]),
'J': [0, 2, 3],
}

init_plan2 = np.array([4,0,5,2,0,0,0,0])
solve(dataset2, init_plan2)



 Initial data 


Initial plan: [4 0 5 2 0 0 0 0]
A = [[ 0  0  1  5  2  0 -5 -4]
 [ 1  1 -1  0  1 -1 -1 -1]
 [ 1  1  1  0  1  2  5  8]]
b = [15 -1  9]
B = [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
c = [ 1 -3  4  3  5  6 -2  0]
J_base: [0, 2, 3]
J*: [0, 2, 3]


Iteration 1

c_: [ 1 -3  4  3  5  6 -2  0]
A base: [[ 0  1  5]
 [ 1 -1  0]
 [ 1  1  0]]
c base: [1 4 3]
u: [-0.6  1.2 -2.2]
Delta: [-2.22044605e-16 -4.00000000e+00  0.00000000e+00 -4.44089210e-16
  2.80000000e+00  4.00000000e-01 -1.12000000e+01 -1.64000000e+01]
j0: 7
[ 0  0  0 -4 -1  8]
l: [-3.5  0.  -4.5  1.7  0.   0.   0.   1. ]
theta: [1.14285714 1.11111111        inf        inf]
theta0: 1.1111111111111112
j*: 2
x_new: [0.11111111 0.         0.         3.88888889 0.         0.
 0.         1.11111111]
J_base: [0, 3, 7]
J_ast: [0, 3, 7]


Iteration 2

c_: [ 1. -3.  4.  3.  5.  6. -2.  0.]
A base: [[ 0  5 -4]
 [ 1  0 -1

array([0.        , 0.66666667, 0.        , 4.66666667, 0.        ,
       0.        , 1.66666667, 0.        ])

In [43]:
alphas = [0., 0.1, 0.5, 1., 10.]
dataset3 = {
    'A': np.array([
        [1., 2., -5., 1., 0., 3., 9., 0., 0.],
        [5., 9., 4., 0., 0., -4., 2., 1., 0.],
        [4., 0., 0., 0., 0., 7., -8., 0., 1.],
        [7., 3., 8., 0., 1., 6., 1., 0., 0.],
    ]),
    # 'c': np.zeros(9),
    'd': np.array([4., 5.]),
    'b': np.array([11., 17., 4., 26.]),
    'B': np.array([
        [2., 0., 4., -3., 0., 2., -4., 1., 2.],
        [0., 0., -5., 1., 1., -1., 5., 1., 3.],
    ]),
    # 'D': alphas[4] * np.diag(np.ones(9)),
    'J': [3, 4, 7, 8],
}
init_plan3 = np.array([0., 0., 0., 11., 26., 0., 0., 17., 4.])

In [44]:
solve(dataset3, init_plan3)



 Initial data 


Initial plan: [ 0.  0.  0. 11. 26.  0.  0. 17.  4.]
A = [[ 1.  2. -5.  1.  0.  3.  9.  0.  0.]
 [ 5.  9.  4.  0.  0. -4.  2.  1.  0.]
 [ 4.  0.  0.  0.  0.  7. -8.  0.  1.]
 [ 7.  3.  8.  0.  1.  6.  1.  0.  0.]]
b = [11. 17.  4. 26.]
D = [[  4.   0.   8.  -6.   0.   4.  -8.   2.   4.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  8.   0.  41. -17.  -5.  13. -41.  -1.  -7.]
 [ -6.   0. -17.  10.   1.  -7.  17.  -2.  -3.]
 [  0.   0.  -5.   1.   1.  -1.   5.   1.   3.]
 [  4.   0.  13.  -7.  -1.   5. -13.   1.   1.]
 [ -8.   0. -41.  17.   5. -13.  41.   1.   7.]
 [  2.   0.  -1.  -2.   1.   1.   1.   2.   5.]
 [  4.   0.  -7.  -3.   3.   1.   7.   5.  13.]]
c = [ -8.   0.   9.   7.  -5.  -3.  -9.  -9. -23.]
J_base: [3, 4, 7, 8]
J*: [3, 4, 7, 8]


Iteration 1

c_: [ -24.    0. -353.   97.   61.  -85.  353.   49.  159.]
A base: [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
c base: [ 97.  61.  49. 159.]
u: [ -97.  -49. -159.  -61.]
Delta: [-1429.  -

array([0.        , 1.02249775, 1.48945193, 2.68463612, 0.        ,
       1.67518419, 0.96578616, 6.60887691, 0.        ])